# Supernova H5 Notebook 
This notebook aims to plot different properties such as temperature and density as well as composition throughout a pre supernova and post supernova star using HDF5 files. HDF5 files have the ability of separating data into "groups" which here are the pre supernova (PreSn/Full) and the explosive (Expl) models. The explosive model is further separated into the "Expl/Runs" and "Expl/Full" groups which contain temporal data for individual zones and the final data after shock propagation, respectfully.  

## Getting the data

First we obtain the H5 file, which contains data on the presupernova star, the individual zones during the supernova, and the post explosion star. Then, specific properties are extracted, which can be modified for the users needs. This will take a few minutes. 

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.animation import FuncAnimation as anim

!curl -o out.h5.gz -J -L https://osf.io/73zd2/download
!gunzip out.h5.gz
!{sys.executable} -m pip install --quiet wnutils

import wnutils.h5 as w5
h5=w5.H5('out.h5')

#Zone '200' vs. time during shockwave propagation, to vary the zone just change 'Expl/Runs/#' to the zone # of interest
props_run=h5.get_group_properties_in_zones_as_floats('Expl/Runs/200',['time','t9','rho'])

#Post supernova star (After shockwave propogation)
props_postsn=h5.get_group_properties_in_zones_as_floats('Expl/Full',['t9','shock t9','t9_0','mass below','shock time'])

Next, we obtain the labels and index IDs for isotopes of interest. Isotopes are specified in the "species" variable by the format: 'symbolA', where symbol is the element symbol and A is the mass number of the isotope. Here we look at $^{15}$N and $^{18}$O, but the species list can be modified to whatever species are of interest to you:

In [ ]:
species = ['n15','o18']
lnames=h5.get_latex_names(species)
indices={}
for sp in species:
  indices[sp]=(h5.get_nuclide_data())[sp]['index']
#mass fractions in zone '200' vs. time during shockwave propagation
mfracs_run=h5.get_group_mass_fractions('Expl/Runs/200')

### Functions 

The following are two functions that will be used later in the program, one for creating the animation of the shockwave and the other for plotting mass fractions throughout stars. 

In [ ]:
#function for animating the shockwave
#Argument: 
#     props - dict of post supernova properties includeint 't9','t9_0','shock t9', and 'mass below'
#     makeanim - whether or not the cell will execute the code, set to false by default
def make_shock_anim(props, makeanim):
  if makeanim:
      from IPython.display import HTML
      from matplotlib.animation import FuncAnimation

      #x - constant, will be the 'mass below' value throughout the star
      #y - varies, contains the temperature of each zone throughout the star at each time step during the propagation of the shock
      x=[]
      y=[]

      #build the x array
      for m in range(len(props_postsn['mass below'])):
        x.append(props_postsn['mass below'][m]/1.9891e33)

      #populate y with initial temperatures 
      for t in props_postsn['t9_0']:
        y.append(t)

      #create a figure for the animation to be written on
      fig,ax=plt.subplots()
      ax.set_xlim([1.4,8])
      ax.set_ylim([-1,9])
      ax.set_xlabel('Mass Below')
      ax.set_ylabel('Temperature ($10^9$K)')
      line, =ax.plot(x,y)


      n_tot = len(props_postsn['shock time'])
      i=np.linspace(0,n_tot-1,n_tot)


      #the function that changes zone temperatures for each time step (animates the plot)
      def shockmovie(t):
        i=int(t)
        y[i]=props_postsn['shock t9'][i]
        for j in range(0,i):
          y[j]=props_postsn['shock t9'][i]
        line.set_xdata(x)
        line.set_ydata(y)
        return line,

      animation=FuncAnimation(fig, func=shockmovie, frames=i, interval=40)
      display(HTML(animation.to_jshtml()))
      plt.close()


#function for plotting isotopic ratios 
#Arguments: 
#    num - the numerator isotope (str) 
#    den - is the denominator isotope (str)
def plot_pre_post_ratios(num,den):

  mfrac=[num,den] 
  ind={}
  for sp in mfrac:
    ind[sp]=(h5.get_nuclide_data())[sp]['index']
  lanames=h5.get_latex_names(mfrac)
  #Calculate the isotopic ratios 
  pre_sn_frac=h5.get_group_mass_fractions("PreSn/Full")[:,ind[mfrac[0]]]/h5.get_group_mass_fractions("PreSn/Full")[:,ind[mfrac[1]]]
  post_sn_frac=h5.get_group_mass_fractions("Expl/Full")[:,ind[mfrac[0]]]/h5.get_group_mass_fractions("Expl/Full")[:,ind[mfrac[1]]]

  plt.plot(props_postsn['mass below']/1.9891e33,pre_sn_frac,label='Pre Supernova')
  plt.plot(props_postsn['mass below']/1.9891e33,post_sn_frac,label='Post Supernova')
  plt.yscale('log')
  plt.xlabel('Mass Below')
  plt.ylabel(lanames[mfrac[0]] +'/' + lanames[mfrac[1]])
  plt.legend()
  plt.xlim([1.4,10])
  plt.show()

## Plotting a property vs. interior mass coordinate (Pre-supernova star)

The following pulls data from the props_presn dictionary to plot the temperature and density vs. interior mass coordinate in a pre-supernova star:

In [ ]:
h5.plot_group_properties_vs_property('PreSn/Full', 'mass below', ['t9','rho'],
                                     xfactor = 1.989e33, 
                                     yscale = 'log',
                                     xlabel = 'Mass Below ($M_\odot$)',
                                     plotParams = [{'label':r'T$_9$'},{'label':r'$\rho$'}]
                                    )

## Plot mass fractions vs. interior mass coordinate (Pre-supernova star)

Now we will plot the mass fraction of isotopes of interest throughout the pre-supernova star. The isotopes of interest should have been defined earlier in the 'getting the data' section of this notebook. The following plots the mass fraction vs. interior mass coordinate. 

In [ ]:
h5.plot_group_mass_fractions_vs_property('PreSn/Full','mass below', species, 
                                         yscale = 'log',
                                         xfactor = 1.989e33,
                                         use_latex_names = True,
                                         xlabel = 'Mass Below ($M_\odot$)',
                                         ylabel = 'Mass Fraction')

## Plotting supernova quantities during shockwave propagation

The following looks at one zone of the star during the propagation of the shockwave throughout the star. To plot the temperature vs. time:

In [ ]:
plt.plot(props_run['time'],props_run['t9'])
plt.xscale('log')
plt.xlim([1e-5,100])
plt.xlabel('Time(s)')
plt.ylabel('$T(10^9K)$')
plt.title('Temperature spike from a supernova shockwave')
plt.show()

Next we want to examine how the abundance of isotopes change in a zone of the star as a result of the supernova shockwave:

In [ ]:
for sp in species:
  plt.plot(props_run['time'],mfracs_run[:,indices[sp]],label=lnames[sp])
plt.xlabel('Time(s)')
plt.xscale('log')
plt.yscale('log')
plt.ylabel('Mass Fraction')
plt.xlim([1e-5,100])
plt.title('Abundances in zone 200 during shockwave propagation')
plt.legend()
plt.show()

## Visualizing the shock

From the well known Rankine-Hugoniot relations the shock speed is given by:
\begin{gather}
v_s = \Big[ \frac{p_0}{\rho_0}\Big(\frac{p}{p_0}-1\Big)\Big(1-\frac{\rho_0}{\rho}\Big)^{-1} \Big]^{1/2}
\end{gather}
which in the frame of the shock is the speed at which the unshocked material moves towards the shock. The shocked material moves away from the shock with the speed:
\begin{gather}
v = \frac{\rho_0 v_s}{\rho}
\end{gather}

Then, in the frame of the star, the speed of material after it is shocked is given by:
\begin{gather}
v = v_s\Big(1-\frac{\rho_0}{\rho}\Big)
\end{gather}

With this we can plot the velocity of each zone of the star after being shocked. We first obtain the necessary properties from the supernova model:

In [ ]:
shock_props=h5.get_group_properties_in_zones_as_floats('Expl/Full', ['mass below','shock speed','rho_0','shock rho'])

Now we calculate and plot the shock speed and speed of the shocked material:

In [ ]:
v_s = shock_props['shock_speed']/1e5
rho = shock_props['shock rho']
rho_0 = shock_props['rho_0']
mass_below = shock_props['mass below']/1.9891e33

v = v_s*(1-rho_0/rho)

fix, ax = plt.subplots()
ax.plot(mass_below, v_s, label = r'$v_s$')
ax.plot(mass_below, v, label = r'v')
ax.set_xlabel(r'Mass Below (M$_\odot$)')
ax.set_ylabel('Velocity (km/s)')
ax.legend()

plt.show()

The massive peak in shock speed followed by the rapid decrease allows for the formation of helium wall.

## Animating the shockwave

The following animates the temperature spike from the shock wave propigation throughout the star. Behind the shock is approximated as an isothermal ball, so after the material is shocked it's temperature evolves with time as the shock temperature. To display the animation set "makeanim=True." The animation will take a minute to be created.  

In [ ]:
makeanim = False
make_shock_anim(props_postsn, makeanim)

## Plotting mass fractions vs. properties post supernova

The composition of the star is altered by the explosive nucleosynthesis. Some general features of the convective core burning stages will be preserved such as the dominance of oxygen in the inner regions, but the composition of specific isotopes will be greatly altered in some cases. To explore the final composition of your isotopes post supernova run the following cell:

In [ ]:
h5.plot_group_mass_fractions_vs_property('Expl/Full','mass below', species, 
                                         yscale = 'log',
                                         xfactor = 1.989e33,
                                         use_latex_names = True,
                                         xlabel = 'Mass Below ($M_\odot$)',
                                         ylabel = 'Mass Fraction')

In the section that follows, a comparison between the pre and post supernova compositions is done.

## Plotting isotopic ratios pre and post supernova

To examine how an isotopic ratio changes vs. interior mass unit for both the pre and post supernova simply call the function 'plot_pre_post_ratios' with the arguments being the numerator and denominator isotope. 

In [ ]:
plot_pre_post_ratios('o16','o18')